In [ ]:
!python -m spacy download en
!pip install transformers
!pip install torch
!pip install tensorflow
!python -m spacy download en_core_web_lg

In [2]:
import numpy as np
import networkx as nx
import spacy
from summa import keywords
from summa.summarizer import summarize

# Laden des Spacy-Modells
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM, PegasusForConditionalGeneration, PegasusTokenizer, AutoTokenizer
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import math
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm
import random
from transformers import pipeline
import torch

In [18]:
df_train=pd.read_csv('data/data_train_with_features.csv')
df_train = df_train.drop('Unnamed: 0', axis=1)
df_train.head()

,classification,text,sentences,num_sentences,words,num_words,lemmas,stops,num_stops
0,Scientific,the interest in anchoring phenomena and phenom...,['the interest in anchoring phenomena and phen...,222,"['interest', 'anchoring', 'phenomena', 'phenom...",3533,"['interest', 'anchor', 'phenomenon', 'phenomen...","[the, in, and, in, has, been, by, their, in, a...",2426
1,Scientific,"galaxy clusters , as the largest peaks in the ...","['galaxy clusters , as the largest peaks in th...",372,"['galaxy', 'clusters', 'largest', 'peaks', 'co...",8982,"['galaxy', 'cluster', 'large', 'peak', 'cosmic...","[as, the, in, the, an, in, and, in, for, and, ...",5768
2,Scientific,quantum correlations between components of a s...,['quantum correlations between components of a...,138,"['quantum', 'correlations', 'components', 'sys...",2451,"['quantum', 'correlation', 'component', 'syste...","[between, of, a, or, are, on, the, of, whole, ...",1780
3,Scientific,methanol masers are often found in star - form...,['methanol masers are often found in star - fo...,141,"['methanol', 'masers', 'found', 'star', 'formi...",3232,"['methanol', 'maser', 'find', 'star', 'form', ...","[are, often, in, are, two, of, to, i, most, th...",1940
4,Scientific,interdisciplinary research has recently gained...,['interdisciplinary research has recently gain...,88,"['interdisciplinary', 'research', 'recently', ...",2695,"['interdisciplinary', 'research', 'recently', ...","[has, a, in, the, of, to, the, in, was, to, tw...",1997


In [ ]:
!python -m spacy download en_core_web_lg

In [9]:





def textrank_extractive(text, compression_rate=0.5):
    # Tokenisierung
    nlp = spacy.load("en_core_web_lg")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    # Speichern der Spacy-Dokumente der Sätze für spätere Verwendung
    sentence_docs = [nlp(sentence) for sentence in sentences]

    # Extrahiere Schlüsselsätze mit TextRank
    num_sentences = max(1, int(len(sentences) * compression_rate))
    extracted_sentences = summarize(text, words=num_sentences, split=True)

    # Erzeuge eine Matrix mit den Ähnlichkeiten zwischen den Sätzen
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for i, doc_i in enumerate(sentence_docs):
        for j, doc_j in enumerate(sentence_docs):
            similarity = similarity_function(doc_i, doc_j)
            similarity_matrix[i, j] = similarity

    # Konstruiere einen Graphen mit den Sätzen als Knoten und den Ähnlichkeiten als Kanten
    graph = nx.from_numpy_array(similarity_matrix)

    # Berechne den TextRank-Score für jeden Satz
    scores = nx.pagerank(graph)

    # Wähle die besten Sätze basierend auf ihren TextRank-Scores aus
    top_sentences = sorted(scores, key=scores.get, reverse=True)[:num_sentences]

    # Sortiere die ausgewählten Sätze nach ihrer Position im Text
    top_sentences = sorted(top_sentences)

    # Gebe die extrahierten Schlüsselsätze zurück
    extracted_sentences = [sentences[index] for index in top_sentences]
    return extracted_sentences


def similarity_function(doc1, doc2):
    # Berechne die Cosinus-Ähnlichkeit zwischen den beiden Dokumenten
    similarity = doc1.similarity(doc2)
    return similarity




def compression_ratio(text, summary):
    # Berechne das Verhältnis der Anzahl der Wörter in der Zusammenfassung zur Anzahl der Wörter im Ausgangstext
    num_words_text = len(text.split())
    num_words_summary = len(summary.split())
    ratio = num_words_summary / num_words_text
    return ratio



def compression(text, compression_rate):
    max_iterations = 20
    iterations = 0
    #compression_rate -= 0.05
    
    extracted = textrank_extractive(text, compression_rate)
    summary = ' '.join(extracted)
    compression_rate_renwed = compression_rate


    while compression_ratio(text, summary) < compression_rate and iterations < max_iterations:
        iterations += 1
        compression_rate_renwed += 0.05
        if compression_rate_renwed > 1:
            compression_rate_renwed = 1
        extracted = textrank_extractive(text, compression_rate=compression_rate_renwed)
        summary = ' '.join(extracted)
    return summary




In [10]:
text = "Deutschland ([ˈdɔɪ̯t͡ʃlant] Audiodatei abspielen; Vollform des Staatennamens seit 1949: Bundesrepublik Deutschland) ist ein Bundesstaat in Mitteleuropa.[6] Er hat 16 Bundesländer und ist als freiheitlich-demokratischer und sozialer Rechtsstaat verfasst. Die 1949 gegründete Bundesrepublik Deutschland stellt die jüngste Ausprägung des 1871 erstmals begründeten deutschen Nationalstaates dar. Bundeshauptstadt und Regierungssitz ist Berlin. Deutschland grenzt an neun Staaten, es hat Anteil an der Nord- und Ostsee im Norden sowie dem Bodensee und den Alpen im Süden. Es liegt in der gemäßigten Klimazone und verfügt über 16 National- und mehr als 100 Naturparks. Das heutige Deutschland hat circa 84,4 Millionen Einwohner und zählt bei einer Fläche von 357.588 Quadratkilometern mit durchschnittlich 236 Einwohnern pro Quadratkilometer zu den dicht besiedelten Flächenstaaten. Die bevölkerungsreichste deutsche Stadt ist Berlin; weitere Metropolen mit mehr als einer Million Einwohnern sind Hamburg, München und Köln; der größte Ballungsraum ist das Ruhrgebiet. Frankfurt am Main ist als europäisches Finanzzentrum von globaler Bedeutung. Die Geburtenrate liegt bei 1,58 Kindern pro Frau (2021).[7] Auf dem Gebiet Deutschlands ist die Anwesenheit von Menschen vor 600.000 Jahren durch Funde des Homo heidelbergensis sowie zahlreiche prähistorischer Kunstwerke aus der späteren Altsteinzeit nachgewiesen. Während der Jungsteinzeit, um 5600 v. Chr., wanderten die ersten Bauern mitsamt Vieh und Saatgut aus dem Nahen Osten ein. Seit der Antike ist die lateinische Bezeichnung Germania für das Siedlungsgebiet der Germanen bekannt. Das ab dem 10. Jahrhundert bestehende römisch-deutsche Reich, das aus vielen Herrschaftsgebieten bestand, war wie der 1815 ins Leben gerufene Deutsche Bund und die liberale demokratische Bewegung Vorläufer des späteren deutschen Gesamtstaates, der 1871 als Deutsches Reich gegründet wurde. Die rasche Entwicklung vom Agrar- zum Industriestaat vollzog sich während der Gründerzeit in der zweiten Hälfte des 19. Jahrhunderts. Nach dem Ersten Weltkrieg wurde 1918 die Monarchie abgeschafft und die demokratische Weimarer Republik konstituiert. Ab 1933 führte die nationalsozialistische Diktatur zu politischer und rassistischer Verfolgung und gipfelte in der Ermordung von sechs Millionen Juden und Angehörigen anderer Minderheiten wie Sinti und Roma. Der vom NS-Staat 1939 begonnene Zweite Weltkrieg endete 1945 mit der Niederlage der Achsenmächte. Das von den Siegermächten besetzte Land wurde 1949 geteilt, nachdem bereits 1945 seine Ostgebiete teils unter polnische, teils sowjetische Verwaltungshoheit gestellt worden waren. Der Gründung der Bundesrepublik als demokratischer westdeutscher Teilstaat mit Westbindung am 23. Mai 1949 folgte die Gründung der sozialistischen DDR am 7. Oktober 1949 als ostdeutscher Teilstaat unter sowjetischer Hegemonie. Die innerdeutsche Grenze war nach dem Berliner Mauerbau (ab 13. August 1961) abgeriegelt. Nach der friedlichen Revolution in der DDR 1989 erfolgte die Lösung der deutschen Frage durch die Wiedervereinigung beider Landesteile am 3. Oktober 1990, womit auch die Außengrenzen Deutschlands als endgültig anerkannt wurden. Durch den Beitritt der fünf ostdeutschen Länder sowie die Wiedervereinigung von Ost- und West-Berlin zur heutigen Bundeshauptstadt zählt die Bundesrepublik Deutschland seit 1990 sechzehn Bundesländer. Deutschland ist Gründungsmitglied der Europäischen Union und ihrer Vorgänger (Römische Verträge 1957) sowie deren bevölkerungsreichstes Land. Mit 18 anderen EU-Mitgliedstaaten bildet es eine Währungsunion, die Eurozone. Es ist Mitglied der UN, der OECD, der OSZE, der NATO, der G7, der G20 und des Europarates. Die Vereinten Nationen unterhalten seit 1951 ihren deutschen Sitz in Bonn („UNO-Stadt“).[8] Die Bundesrepublik Deutschland gilt als einer der politisch einflussreichsten Staaten Europas und ist ein gesuchtes Partnerland auf globaler Ebene.[9] Gemessen am Bruttoinlandsprodukt ist das marktwirtschaftlich organisierte Deutschland die größte Volkswirtschaft Europas und die viertgrößte der Welt.[10] 2016 war es die drittgrößte Export- und Importnation.[11] Es ist eine Informations- und Wissensgesellschaft. Automatisierung, Digitalisierung und Disruption prägen die innovative deutsche Industrieentwicklung. Die Steigerung der Qualität des deutschen Bildungssystems und die nachhaltige Entwicklung des Landes gelten als zentrale Aufgaben der Standortpolitik. Gemäß dem Index der menschlichen Entwicklung zählt Deutschland zu den sehr hoch entwickelten Ländern.[12][13] Muttersprache der Bevölkerungsmehrheit ist Deutsch. Daneben gibt es Regional- und Minderheitensprachen und sowohl Deutsche als auch Migranten mit anderen Muttersprachen, von denen die bedeutendsten Türkisch und Russisch sind.[14] Bedeutendste Fremdsprache ist Englisch, das in allen Bundesländern in der Schule gelehrt wird. Die Kultur Deutschlands ist vielfältig und wird neben zahlreichen Traditionen, Institutionen und Veranstaltungen beispielsweise in der Auszeichnung als UNESCO-Welterbe in Deutschland, in Kulturdenkmälern und als immaterielles Kulturerbe erfasst und gewürdigt."

In [14]:
print(compression(text,0.3))
print(compression_ratio(text, compression(text,0.3)))

/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_51925/739091902.py:40: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = doc1.similarity(doc2)


Bundeshauptstadt und Regierungssitz ist Berlin. Es liegt in der gemäßigten Klimazone und verfügt über 16 National- und mehr als 100 Naturparks. Die bevölkerungsreichste deutsche Stadt ist Berlin; weitere Metropolen mit mehr als einer Million Einwohnern sind Hamburg, München und Köln; der größte Ballungsraum ist das Ruhrgebiet. Die Geburtenrate liegt bei 1,58 Kindern pro Frau (2021).[7] Auf dem Gebiet Deutschlands ist die Anwesenheit von Menschen vor 600.000 Jahren durch Funde des Homo heidelbergensis sowie zahlreiche prähistorischer Kunstwerke aus der späteren Altsteinzeit nachgewiesen. Seit der Antike ist die lateinische Bezeichnung Germania für das Siedlungsgebiet der Germanen bekannt. Jahrhundert bestehende römisch-deutsche Reich, das aus vielen Herrschaftsgebieten bestand, war wie der 1815 ins Leben gerufene Deutsche Bund und die liberale demokratische Bewegung Vorläufer des späteren deutschen Gesamtstaates, der 1871 als Deutsches Reich gegründet wurde. Die rasche Entwicklung vom A

In [43]:

model_name='facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline("text2text-generation", model=model_name)

In [44]:
def token_count(text):
    tokens = text.split()
    return len(tokens)


def adjust_length(text):
    length = token_count(text)
    if length < 80:
        min_length = length + int(length * 0.05)
        max_length = min_length + 50
    elif length < 100:
        min_length = length + int(length * 0.3)
        max_length = min_length + 100
    else:
        min_length = math.ceil(length / 50) * 70
        max_length = min_length + 100
    return min_length, max_length

In [45]:
def batch_sent(sentenc,splitt=180,split='. '):
    sentences = sentenc.split(split)
    # Erstellen Sie Batches von Sätzen, die weniger als 1024 Tokens enthalten
    batches = []
    batch = []
    batch_len = 0
    for sentence in sentences:
        
        sentence_len = len(tokenizer.tokenize(sentence))
        if sentence_len + batch_len > splitt:
            if sentence_len < splitt:  # überspringen Sie Sätze, die länger als 256 Tokens sind
                batches.append(batch)
                batch = [sentence]
                batch_len = sentence_len
            # wenn ein Satz alleine 1024 Tokens überschreitet, wird er übersprungen
        else:
            batch.append(sentence)
            batch_len += sentence_len
    batches.append(batch)
    return batches

In [70]:
def paraphrase_of_text(df_s,text_name='text',komp_name='reduction_multiplier',split='. '):

    df_summary_testing = pd.DataFrame(columns=['Zusammenfassung','Min_Kompressionsrate', 'Max_Kompressionsrate', 'Endgueltige_Kompressionsrate','länge Zusammenfassung','länge Ausgangstext'])
    # Teilen Sie den Text in Sätze --> für tatsächliche umsetzubng
    soplitting=True
    for _, row in tqdm(df_s.iterrows(), total=df_s.shape[0]):
     
        text = row[text_name]
        komp = row[komp_name]
        text_gesamt_list=[]

        for batch in tqdm(batch_sent(text,split=split), desc='Verarbeite Batches'):
            # Zusammenfügen der Sätze in einem Batch
            batch_text = '. '.join(batch)
            min_length_test, max_length_test = adjust_length(batch_text)
            ext_summary=summarizer(batch_text, max_length=int(round(max_length_test*1,0)), min_length=int(round(min_length_test*1,0)),length_penalty=100,num_beams=2)
            # Erstellen Sie einen DataFrame für die aktuellen Ergebnisse
           
            text_gesamt_list.append(ext_summary[0]['generated_text'])
        text_gesamt = '. '.join(text_gesamt_list)
        actual_compression_rate = len(text_gesamt.split(' '))/len(text.split(' '))*100
      
        df_current = pd.DataFrame({
            'Zusammenfassung': [text_gesamt],
            'Min_Kompressionsrate': [min_length_test],
            'Max_Kompressionsrate': [max_length_test],
            'Endgueltige_Kompressionsrate': [actual_compression_rate],
            'länge Zusammenfassung': [len(text_gesamt.split(' '))],
            'länge Ausgangstext': [len(text.split(' '))]
        
        })
       
        # Fügen Sie die Daten zum DataFrame hinzu
        df_summary_testing = pd.concat([df_summary_testing, df_current], ignore_index=True)
      
    return df_summary_testing


In [ ]:
print(compression(text,0.2))
print(compression_ratio(text, compression(text,0.2)))


In [27]:
def calculate_compression(df, total_tokens_col, current_tokens_col, desired_compression_rate):
    df['current_compression_rate'] = df[current_tokens_col] / df[total_tokens_col]
    df['compression_difference'] = df[desired_compression_rate] - df['current_compression_rate']
    df['reduction_multiplier'] = df[desired_compression_rate] / df['current_compression_rate']
    return df

In [ ]:
df_test=calculate_compression(df_test, 'total_tokens', 'token_count', desired_compression_rate)

In [ ]:
df_summary=paraphrase_of_text(df_test[['text','reduction_multiplier']])

In [68]:

def text_rank_algo(df):
    df_return = pd.DataFrame(columns=['text','text_rank_text','tokens_gesamt', 'token_text_rank','desired_compression_rate','text_rank_compression_rate'])
    # Schleife zur Generierung der zufälligen Werte
    for index, row in tqdm(df.iterrows()):
        text = row["text"]
        random.seed(10)
        random_value = round(random.uniform(0.2, 0.8), 2)  # Zufälliger Wert zwischen 0.2 und 0.8 auf zwei Stellen nach dem Komma begrenzt
        text_rank_text= compression(text,random_value)
        compression_ratio_value=compression_ratio(text, compression(text,random_value))
        df_current = pd.DataFrame({
                'text':[text],
                'text_rank_text': [text_rank_text],
                'tokens_gesamt': [len(text.split(' '))],
                'token_text_rank': [len(text_rank_text.split(' '))],
                'desired_compression_rate': [random_value],
                'text_rank_compression_rate': [compression_ratio_value]
            })
            # Fügen Sie die Daten zum DataFrame hinzu
        df_return = pd.concat([df_return, df_current], ignore_index=True)
    return df_return


In [49]:
def execute_text_gen(df,split='. '):
    rank_df=text_rank_algo(df)
    df_zwischen=calculate_compression(rank_df, 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate')
    df_sum=paraphrase_of_text(df_zwischen[['text_rank_text','reduction_multiplier']],text_name='text_rank_text',split=split)
    merged_df = pd.concat([df_sum, df_zwischen], axis=1)
    merged_df['ent_com_rate']=merged_df['länge Zusammenfassung'] / merged_df['tokens_gesamt']
    return merged_df

In [54]:
df_train.classification.unique()


array(['Scientific', 'news', 'reviews', 'story'], dtype=object)

In [58]:
df_news=df_train[df_train.classification=='news'].reset_index(drop=True)[['classification','text']][0:10]

In [73]:
df_2=execute_text_gen(df_news,split='. ')

10it [01:10,  7.10s/it]
100%|██████████| 10/10 [00:53<00:00,  5.33s/it]


In [72]:
df[['Zusammenfassung', 'Min_Kompressionsrate', 'Max_Kompressionsrate',
       'länge Zusammenfassung',
       'text','tokens_gesamt',
       'token_text_rank', 'desired_compression_rate',
       'text_rank_compression_rate', 'current_compression_rate',
      'reduction_multiplier', 'ent_com_rate']]

,Zusammenfassung,Min_Kompressionsrate,Max_Kompressionsrate,länge Zusammenfassung,text,tokens_gesamt,token_text_rank,desired_compression_rate,text_rank_compression_rate,current_compression_rate,reduction_multiplier,ent_com_rate
0,The Administration of Union Territory Daman an...,34,84,33,The Administration of Union Territory Daman an...,60,33,0.54,0.550000,0.55,0.981818,0.55
1,Malaika Arora slammed an Instagram user who tr...,40,90,39,Malaika Arora slammed an Instagram user who tr...,60,39,0.54,0.650000,0.65,0.830769,0.65
2,Bihar Health Minister defined virgin as being ...,36,86,34,The Indira Gandhi Institute of Medical Science...,60,35,0.54,0.583333,0.583333,0.925714,0.566667
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,42,92,22,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,60,40,0.54,0.666667,0.666667,0.81,0.366667
4,Hotels in Maharashtra will train their staff t...,53,103,38,Hotels in Maharashtra will train their staff t...,60,51,0.54,0.850000,0.85,0.635294,0.633333
5,"Man, 32, was found hanging inside the washroom...",50,100,30,A 32-year-old man on Wednesday was found hangi...,60,48,0.54,0.800000,0.8,0.675,0.5
6,Delhi High Court reduced the compensation awar...,37,87,35,The Delhi High Court reduced the compensation ...,61,36,0.54,0.600000,0.590164,0.915,0.57377
7,60-year-old Dalit woman was allegedly lynched ...,63,113,46,A 60-year old Dalit woman was allegedly lynche...,60,60,0.54,1.000000,1.0,0.54,0.766667
8,The Pawan Hans helicopter was flying at a crit...,63,113,40,An inquiry by the Aircraft Accident Investigat...,61,60,0.54,1.000000,0.983607,0.549,0.655738
9,Congress party has opened a bank called 'State...,43,93,30,The Congress party has opened a bank called 'S...,57,41,0.54,0.719298,0.719298,0.750732,0.526316


In [75]:
df_2[['Zusammenfassung', 'Min_Kompressionsrate', 'Max_Kompressionsrate',
       'länge Zusammenfassung',
       'text','tokens_gesamt',
       'token_text_rank', 'desired_compression_rate',
       'text_rank_compression_rate', 'current_compression_rate',
      'reduction_multiplier', 'ent_com_rate']][3:4]

,Zusammenfassung,Min_Kompressionsrate,Max_Kompressionsrate,länge Zusammenfassung,text,tokens_gesamt,token_text_rank,desired_compression_rate,text_rank_compression_rate,current_compression_rate,reduction_multiplier,ent_com_rate
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,42,92,22,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,60,40,0.54,0.666667,0.666667,0.81,0.366667


In [78]:
df_2['Zusammenfassung'][3:4][3]

'Lashkar-e-Taiba\'s Kashmir commander Abu Dujana was killed by security forces. "Hum nikley they shaheed hone (had left home for martyrdom)," he added.'

In [79]:
df_2['text'][3:4][3]

'Lashkar-e-Taiba\'s Kashmir commander Abu Dujana, who was killed by security forces, said "Kabhi hum aage, kabhi aap, aaj aapne pakad liya, mubarak ho aapko (Today you caught me. Congratulations)" after being caught. He added that he won\'t surrender, and whatever is in his fate will happen to him. "Hum nikley they shaheed hone (had left home for martyrdom)," he added.'

In [57]:
df_2[['Zusammenfassung', 'Min_Kompressionsrate', 'Max_Kompressionsrate',
       'länge Zusammenfassung',
       'text','tokens_gesamt',
       'token_text_rank', 'desired_compression_rate',
       'text_rank_compression_rate', 'current_compression_rate',
      'reduction_multiplier', 'ent_com_rate']]


,Zusammenfassung,Min_Kompressionsrate,Max_Kompressionsrate,länge Zusammenfassung,text,tokens_gesamt,token_text_rank,desired_compression_rate,text_rank_compression_rate,current_compression_rate,reduction_multiplier,ent_com_rate
0,Flame's mother ran all the way from the icy-fo...,210,310,2746,"PART I\n\nIf you don't like Christmas stories,...",5370,3592,0.46,0.659574,0.668901,0.687695,0.511359
1,The Young Electrician was neither one length n...,110,210,2607,The Railroad Journey was very long and slow. T...,5526,3980,0.43,0.713496,0.720232,0.59703,0.47177
2,The author knew that he was going to have to t...,53,103,2391,His affair that night was prosy. He was intend...,5494,4373,0.51,0.780829,0.795959,0.640736,0.435202
3,"""Enough croakings for the day, Leaky-Tongue!"" ...",210,310,2559,"This is the tale which Jehan Tugluk Khan, a wi...",4270,4005,0.75,0.924485,0.937939,0.799625,0.599297
4,"Stuart McGregor's obsession of horror, of a ce...",16,66,2098,THE fact that the man whom he feared had died ...,3347,3151,0.69,0.927315,0.94144,0.73292,0.62683
5,"""You can compel a Brahmin to sit in the same o...",210,310,830,"IBRAHIM FADLALLAH shrugged bis shoulders: ""You...",1715,1085,0.35,0.630787,0.632653,0.553226,0.483965
6,"The Koh Haji-Lal, the ""Mountains of the Red Pi...",57,107,1928,"BEHIND him the Koh Haji-Lal, the ""Mountains of...",2855,2553,0.64,0.876070,0.894221,0.715707,0.675306
7,"""I was quite out of my head when that fellow f...",68,118,1546,SPEAKING in after years about that period of h...,3935,2292,0.35,0.572784,0.582465,0.600894,0.392884
8,"The West has produced many types of vagabond,-...",75,125,4151,The ease and apparent willingness with which s...,4338,3689,0.73,0.843814,0.850392,0.858428,0.956893
9,The history of this man is somewhat obscured b...,52,102,1338,Along in the 80's there occurred a question of...,2302,1259,0.42,0.540808,0.546916,0.767943,0.581234
